In [19]:
import pandas as pd
import glob
import os
import numpy as np
import math

# pd.options.plotting.backend = "plotly"
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from pint import UnitRegistry

ureg = UnitRegistry()
Q_ = ureg.Quantity

In [20]:
OSX_path = r"/Users/jdtarriela/Documents/GitHub/FDA_Blood_Pump/Core/Cond_5-KW_19M_BP/Residuals/*.out"
win_path = r"C:\Users\Joseph Tarriela\Documents\GitHub\FDA_Blood_Pump\Core\Cond_5-KW_19M_BP\Residuals/*.out"

filenames = [os.path.basename(x) for x in glob.glob(OSX_path)]
file_paths = glob.glob(OSX_path)

sample_time_step = np.arange(1800, 3690, 90).tolist() # arange does not include last number, increment 1 higher

resid_dict = {}
time_sample_dict = {}
for i in enumerate(filenames):
    # pd_temp = pd.read_csv(file_paths[i[0]], header=0, sep=" ")
    resid_dict[i[1]] = pd.read_csv(file_paths[i[0]], header=0, sep=" ")
    time_sample_dict[i[1]] = pd.read_csv(file_paths[i[0]], header=0, sep=" ")
    time_sample_dict[i[1]] = time_sample_dict[i[1]][time_sample_dict[i[1]]['Time Step'].isin(sample_time_step)]

fluid_density = 1035
Hb = 11500
resid_dict['gw-hemolysis.out']['pfHb/Hb'] = np.power(resid_dict['gw-hemolysis.out']['gw-hemolysis']/fluid_density,0.785)
resid_dict['gw-hemolysis.out']['pfHb'] = resid_dict['gw-hemolysis.out']['pfHb/Hb']*Hb
resid_dict['ho-hemolysis.out']['pfHb/Hb'] = np.power(resid_dict['ho-hemolysis.out']['ho-hemolysis']/fluid_density,0.765)
resid_dict['ho-hemolysis.out']['pfHb'] = resid_dict['ho-hemolysis.out']['pfHb/Hb']*Hb
time_sample_dict['gw-hemolysis.out']['pfHb/Hb'] = np.power(resid_dict['gw-hemolysis.out']['gw-hemolysis']/fluid_density,0.785)
time_sample_dict['gw-hemolysis.out']['pfHb'] = resid_dict['gw-hemolysis.out']['pfHb/Hb']*Hb
time_sample_dict['ho-hemolysis.out']['pfHb/Hb'] = np.power(resid_dict['ho-hemolysis.out']['ho-hemolysis']/fluid_density,0.765)
time_sample_dict['ho-hemolysis.out']['pfHb'] = resid_dict['ho-hemolysis.out']['pfHb/Hb']*Hb
time_sample_dict['gw-hemolysis.out']['pfHb-pct-change'] = time_sample_dict['gw-hemolysis.out']['pfHb'].pct_change()
time_sample_dict['gw-hemolysis.out']['pfHb-pct-change'] = time_sample_dict['gw-hemolysis.out']['pfHb-pct-change']*100
time_sample_dict['ho-hemolysis.out']['pfHb-pct-change'] = time_sample_dict['ho-hemolysis.out']['pfHb'].pct_change()
time_sample_dict['ho-hemolysis.out']['pfHb-pct-change'] = time_sample_dict['ho-hemolysis.out']['pfHb-pct-change']*100


resid_dict['Pressure_Head'] = resid_dict['pressure-in.out'].copy()
resid_dict['Pressure_Head']['p-out'] = resid_dict['pressure-out.out']['p-out']
resid_dict['Pressure_Head']['p-head-mmHg'] = (resid_dict['Pressure_Head']['p-out']-resid_dict['Pressure_Head']['p-in'])/(13595.1*9.80665)*1000

time_sample_dict['Pressure_Head'] = time_sample_dict['pressure-in.out'].copy()
time_sample_dict['Pressure_Head']['p-out'] = time_sample_dict['pressure-out.out']['p-out']
time_sample_dict['Pressure_Head']['p-head-mmHg'] = (time_sample_dict['Pressure_Head']['p-out']-time_sample_dict['Pressure_Head']['p-in'])/(13595.1*9.80665)*1000


In [21]:
# # rev_complete = 5
# # case = {'rpm': Q_(3500, 'rpm'),
# #         'blades': 4,
# #         'timestep': Q_(3/280000, 'sec')}
# #
# # rps = case['rpm'].to('rps')  #revolutions per second
# # sec_per_rev = 1 / rps * Q_(1, 'revolutions')
# # sec_per_rev = sec_per_rev.to('sec')
# # sec_per_blade_pass = sec_per_rev / case['blades']
# # step_per_blade_pass = sec_per_blade_pass / case['timestep']
# # step_per_rev = step_per_blade_pass * 4
# #
# # ### Change to suit bin size--
# # ### Either per blade pass or per revolution for smoothing
# # bin_size = int((step_per_rev * rev_complete) / step_per_blade_pass)
# #
# # print(f'For Specified Timestep: {case["timestep"]} \n'
# #       f'Seconds per revolution: {sec_per_rev} \n'
# #       f'Seconds per blade pass: {sec_per_blade_pass} \n'
# #       f'Time steps per blade pass: {step_per_blade_pass} \n'
# #       f'Time steps per rotation: {step_per_blade_pass * 4}\n\n'
# #       f'One bin per blade pass\n\n'
# #       f'Number of bins: {bin_size}\n\n')
# #
# # bin_df = residual_df.copy()
# # # Use df.copy(): indexing a DataFrame returns a reference to the initial DataFrame.
# # # Thus, changing the subset will change the initial DataFrame.
# # # Thus, you'd want to use the copy if you want to make sure the initial DataFrame shouldn't change.
# #
# # col_list = bin_df.columns[0:].copy()
#
# # bin_df["Bins"] = pd.cut(bin_df[col_list[0]], bins=bin_size)
# # bin_df["bin_centres"] = bin_df["Bins"].apply(lambda x: x.mid)
# # bin_df['Counts'] = bin_df.groupby(['Bins'])[col_list[0]].transform('count')
# # bin_df['Bin Variance'] = bin_df.groupby(['Bins'])[col_list[0]].transform('var')
# # for variable in enumerate(resid_col_list):
# #     # print(variable)
# #     if variable[0] == 0:
# #         # skip first column "Time_step"
# #         pass
# #     else:
# #         # print(variable)
# #         col_bin_var_mean = variable[1] + "_bin_mean"
# #         col_bin_std = variable[1] + "_std"
# #         col_bin_sem = variable[1] + "_sem"
# #         bin_df[col_bin_var_mean] = bin_df.groupby('Bins')[col_list[variable[0]]].transform('mean')
# #         bin_df[col_bin_std] = bin_df.groupby('Bins')[col_list[variable[0]]].transform('std')
# #         bin_df[col_bin_sem] = bin_df.groupby('Bins')[col_list[variable[0]]].transform('sem')
# #
# # bin_df = bin_df.drop_duplicates(subset="Bins")
# # bin_df = bin_df.reset_index()
# # bin_df = bin_df.drop(resid_col_list, axis=1)
#
# for i in enumerate(filenames):
#     col_list = resid_dict[i[1]].columns[0:].copy()
#
#     resid_dict[i[1]]["Bins"] = pd.cut(bin_df[col_list[0]], bins=bin_size)
#     resid_dict[i[1]]["bin_centres"] = bin_df["Bins"].apply(lambda x: x.mid)
#     resid_dict[i[1]]['Counts'] = bin_df.groupby(['Bins'])[col_list[0]].transform('count')
#     resid_dict[i[1]]['Bin Variance'] = bin_df.groupby(['Bins'])[col_list[0]].transform('var')


In [22]:
color10_16 = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880', '#FF97FF',
              '#FECB52']
scatter_opacity = 0.7
mean_opacity = scatter_opacity

fig = make_subplots(
    rows=5,
    cols=1,
    shared_xaxes=False,
    x_title="Time Step",
    y_title="Damage Index")
# subplot_titles=("Point A", "Quadrant 2"))

fig.add_trace(go.Scatter(x=resid_dict['point-GW-a.out']['Time Step'],
                         y=resid_dict['point-GW-a.out']['point-gw-a'],
                         mode='markers',
                         opacity=scatter_opacity,
                         name='GW-A',
                         line=dict(color=color10_16[0])),
              row=1, col=1)
# fig.add_trace(go.Scatter(x=bin_df['bin_centres'],
#                          y=bin_df['point-gw-a_bin_mean'],
#                          mode='markers+lines',
#                          name='Mean +/- STD',
#                          line=dict(color='black'),
#                          opacity=.55,
#                          error_x_array=bin_df['point-gw-a_std'],
#                          showlegend=False),
#               row=1, col=1)

fig.add_trace(go.Scatter(x=resid_dict['point-GW-b.out']['Time Step'],
                         y=resid_dict['point-GW-b.out']['point-gw-b'],
                         mode='markers',
                         opacity=scatter_opacity,
                         name='GW-B',
                         line=dict(color=color10_16[1])),
              row=2, col=1)
# fig.add_trace(go.Scatter(x=bin_df['bin_centres'],
#                          y=bin_df['point-gw-b_bin_mean'],
#                          mode='markers+lines',
#                          name='Mean +/- STD',
#                          line=dict(color='black'),
#                          opacity=.55,
#                          error_x_array=bin_df['point-gw-b_std'],
#                          showlegend=False),
#               row=2, col=1)

fig.add_trace(go.Scatter(x=resid_dict['point-GW-c.out']['Time Step'],
                         y=resid_dict['point-GW-c.out']['point-gw-c'],
                         mode='markers',
                         opacity=scatter_opacity,
                         name='GW-C',
                         line=dict(color=color10_16[2]),
                         showlegend=True),
              row=3, col=1)
# fig.add_trace(go.Scatter(x=bin_df['bin_centres'],
#                          y=bin_df['point-gw-c_bin_mean'],
#                          mode='markers+lines',
#                          name='Mean +/- STD',
#                          line=dict(color='black'),
#                          opacity=.55,
#                          error_x_array=bin_df['point-gw-c_std'],
#                          showlegend=False),
#               row=3, col=1)

fig.add_trace(go.Scatter(x=resid_dict['point-GW-d.out']['Time Step'],
                         y=resid_dict['point-GW-d.out']['point-gw-d'],
                         mode='markers',
                         opacity=scatter_opacity,
                         name='GW-D',
                         line=dict(color=color10_16[3]),
                         showlegend=True),
              row=4, col=1)
# fig.add_trace(go.Scatter(x=bin_df['bin_centres'],
#                          y=bin_df['point-gw-d_bin_mean'],
#                          mode='markers+lines',
#                          name='Mean +/- STD',
#                          line=dict(color='black'),
#                          opacity=.55,
#                          error_x_array=bin_df['point-gw-d_std'],
#                          showlegend=False),
#               row=4, col=1)

fig.add_trace(go.Scatter(x=resid_dict['point-GW-e.out']['Time Step'],
                         y=resid_dict['point-GW-e.out']['point-gw-e'],
                         mode='markers',
                         opacity=scatter_opacity,
                         name='GW-E',
                         line=dict(color=color10_16[4]),
                         showlegend=True),
              row=5, col=1)
# fig.add_trace(go.Scatter(x=bin_df['bin_centres'],
#                          y=bin_df['point-gw-e_bin_mean'],
#                          mode='markers+lines',
#                          name='Mean +/- STD',
#                          line=dict(color='black'),
#                          opacity=.55,
#                          error_x_array=bin_df['point-gw-e_std'],
#                          showlegend=True),
#               row=5, col=1)

fig.update_layout(height=1000, width=500,
                  title_text="GW Damage Index Point Monitors")

fig.show()

In [23]:
color10_16 = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880', '#FF97FF',
              '#FECB52']
fig = make_subplots(
    rows=5,
    cols=1,
    shared_xaxes=False,
    x_title="Time Step",
    y_title="Pressure (Pa)")
# subplot_titles=("Point A", "Quadrant 2"))

fig.add_trace(go.Scatter(x=resid_dict['point-pressure-a.out']['Time Step'],
                         y=resid_dict['point-pressure-a.out']['point-pressure-a'],
                         mode='markers',
                         opacity=scatter_opacity,
                         name='Pressure-A',
                         line=dict(color=color10_16[0])),
              row=1, col=1)
# fig.add_trace(go.Scatter(x=bin_df['bin_centres'],
#                          y=bin_df['point-pressure-a_bin_mean'],
#                          mode='markers+lines',
#                          name='Mean +/- STD',
#                          line=dict(color='black'),
#                          opacity=.55,
#                          error_y_array=bin_df['point-pressure-a_std'],
#                          showlegend=False),
#               row=1, col=1)

fig.add_trace(go.Scatter(x=resid_dict['point-pressure-b.out']['Time Step'],
                         y=resid_dict['point-pressure-b.out']['point-pressure-b'],
                         mode='markers',
                         opacity=scatter_opacity,
                         name='Pressure-B',
                         line=dict(color=color10_16[1])),
              row=2, col=1)
# fig.add_trace(go.Scatter(x=bin_df['bin_centres'],
#                          y=bin_df['point-pressure-b_bin_mean'],
#                          mode='markers+lines',
#                          name='Mean +/- STD',
#                          line=dict(color='black'),
#                          opacity=.55,
#                          error_y_array=bin_df['point-pressure-b_std'],
#                          showlegend=False),
#               row=2, col=1)

fig.add_trace(go.Scatter(x=resid_dict['point-pressure-c.out']['Time Step'],
                         y=resid_dict['point-pressure-c.out']['point-pressure-c'],
                         mode='markers',
                         opacity=scatter_opacity,
                         name='Pressure-C',
                         line=dict(color=color10_16[2]),
                         showlegend=True),
              row=3, col=1)
# fig.add_trace(go.Scatter(x=bin_df['bin_centres'],
#                          y=bin_df['point-pressure-c_bin_mean'],
#                          mode='markers+lines',
#                          name='Mean +/- STD',
#                          line=dict(color='black'),
#                          opacity=.55,
#                          error_y_array=bin_df['point-pressure-c_std'],
#                          showlegend=False),
#               row=3, col=1)

fig.add_trace(go.Scatter(x=resid_dict['point-pressure-d.out']['Time Step'],
                         y=resid_dict['point-pressure-d.out']['point-pressure-d'],
                         mode='markers',
                         opacity=scatter_opacity,
                         name='Pressure-D',
                         line=dict(color=color10_16[3]),
                         showlegend=True),
              row=4, col=1)
# fig.add_trace(go.Scatter(x=bin_df['bin_centres'],
#                          y=bin_df['point-pressure-d_bin_mean'],
#                          mode='markers+lines',
#                          name='Mean +/- STD',
#                          line=dict(color='black'),
#                          opacity=.55,
#                          error_y_array=bin_df['point-pressure-d_std'],
#                          showlegend=False),
#               row=4, col=1)

fig.add_trace(go.Scatter(x=resid_dict['point-pressure-e.out']['Time Step'],
                         y=resid_dict['point-pressure-e.out']['point-pressure-e'],
                         mode='markers',
                         opacity=scatter_opacity,
                         name='Pressure-E',
                         line=dict(color=color10_16[4]),
                         showlegend=True),
              row=5, col=1)
# fig.add_trace(go.Scatter(x=bin_df['bin_centres'],
#                          y=bin_df['point-pressure-e_bin_mean'],
#                          mode='markers+lines',
#                          name='Mean +/- STD',
#                          line=dict(color='black'),
#                          opacity=.55,
#                          error_y_array=bin_df['point-pressure-e_std'],
#                          showlegend=True),
#               row=5, col=1)

fig.update_layout(height=700, width=500,
                  title_text="Pressure Point Monitors")

fig.show()

In [24]:
color10_16 = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880', '#FF97FF',
              '#FECB52']
fig = make_subplots(
    rows=5,
    cols=1,
    shared_xaxes=True,
    x_title="Time Step",
    y_title="Velocity Magnitude (m/s)")
# subplot_titles=("Point A", "Quadrant 2"))

fig.add_trace(go.Scatter(x=resid_dict['point-vel-a.out']['Time Step'],
                         y=resid_dict['point-vel-a.out']['point-vel-a'],
                         mode='markers',
                         opacity=scatter_opacity,
                         name='Velocity-A',
                         line=dict(color=color10_16[0])),
              row=1, col=1)

# fig.add_trace(go.Scatter(x=bin_df['bin_centres'],
#                          y=bin_df['point-vel-a_bin_mean'],
#                          mode='markers+lines',
#                          name='Mean +/- STD',
#                          line=dict(color='black'),
#                          opacity=.55,
#                          error_y_array=bin_df['point-vel-a_std'],
#                          showlegend=False),
#               row=1, col=1)


fig.add_trace(go.Scatter(x=resid_dict['point-vel-b.out']['Time Step'],
                         y=resid_dict['point-vel-b.out']['point-vel-b'],
                         mode='markers',
                         opacity=scatter_opacity,
                         name='Velocity-B',
                         line=dict(color=color10_16[1])),
              row=2, col=1)
# fig.add_trace(go.Scatter(x=bin_df['bin_centres'],
#                          y=bin_df['point-vel-b_bin_mean'],
#                          mode='markers+lines',
#                          name='Mean +/- STD',
#                          line=dict(color='black'),
#                          opacity=.55,
#                          error_y_array=bin_df['point-vel-b_std'],
#                          showlegend=False),
#               row=2, col=1)

fig.add_trace(go.Scatter(x=resid_dict['point-vel-c.out']['Time Step'],
                         y=resid_dict['point-vel-c.out']['point-vel-c'],
                         mode='markers',
                         opacity=scatter_opacity,
                         name='Velocity-C',
                         line=dict(color=color10_16[2]),
                         showlegend=True),
              row=3, col=1)
# fig.add_trace(go.Scatter(x=bin_df['bin_centres'],
#                          y=bin_df['point-vel-c_bin_mean'],
#                          mode='markers+lines',
#                          name='Mean +/- STD',
#                          line=dict(color='black'),
#                          opacity=.55,
#                          error_y_array=bin_df['point-vel-c_std'],
#                          showlegend=False),
#               row=3, col=1)

fig.add_trace(go.Scatter(x=resid_dict['point-vel-d.out']['Time Step'],
                         y=resid_dict['point-vel-d.out']['point-vel-d'],
                         mode='markers',
                         opacity=scatter_opacity,
                         name='Velocity-D',
                         line=dict(color=color10_16[3]),
                         showlegend=True),
              row=4, col=1)
# fig.add_trace(go.Scatter(x=bin_df['bin_centres'],
#                          y=bin_df['point-vel-d_bin_mean'],
#                          mode='markers+lines',
#                          name='Mean +/- STD',
#                          line=dict(color='black'),
#                          opacity=.55,
#                          error_y_array=bin_df['point-vel-d_std'],
#                          showlegend=False),
#               row=4, col=1)

fig.add_trace(go.Scatter(x=resid_dict['point-vel-e.out']['Time Step'],
                         y=resid_dict['point-vel-e.out']['point-vel-e'],
                         mode='markers',
                         opacity=scatter_opacity,
                         name='Velocity-E',
                         line=dict(color=color10_16[4]),
                         showlegend=True),
              row=5, col=1)
# fig.add_trace(go.Scatter(x=bin_df['bin_centres'],
#                          y=bin_df['point-vel-e_bin_mean'],
#                          mode='markers+lines',
#                          name='Mean +/- STD',
#                          line=dict(color='black'),
#                          opacity=.55,
#                          error_y_array=bin_df['point-vel-e_std'],
#                          showlegend=True),
#               row=5, col=1)

fig.update_layout(height=700, width=500,
                  title_text="Velocity Point Monitors")

fig.show()

In [25]:
color10_16 = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880', '#FF97FF',
              '#FECB52']
fig = make_subplots(
    rows=5,
    cols=1,
    shared_xaxes=True,
    x_title="Time Step",
    y_title="Von Mises Stress (Pa)")
# subplot_titles=("Point A", "Quadrant 2"))

fig.add_trace(go.Scatter(x=resid_dict['point-tau_vm-a.out']['Time Step'],
                         y=resid_dict['point-tau_vm-a.out']['point-tau_vm-a'],
                         mode='markers',
                         opacity=scatter_opacity,
                         name='Shear-A',
                         line=dict(color=color10_16[0])),
              row=1, col=1)
# fig.add_trace(go.Scatter(x=bin_df['bin_centres'],
#                          y=bin_df['point-shear-a_bin_mean'],
#                          mode='markers+lines',
#                          name='Mean +/- STD',
#                          line=dict(color='black'),
#                          opacity=.55,
#                          error_y_array=bin_df['point-shear-a_std'],
#                          showlegend=False),
#               row=1, col=1)

fig.add_trace(go.Scatter(x=resid_dict['point-tau_vm-b.out']['Time Step'],
                         y=resid_dict['point-tau_vm-b.out']['point-tau_vm-b'],
                         mode='markers',
                         opacity=scatter_opacity,
                         name='Shear-B',
                         line=dict(color=color10_16[1])),
              row=2, col=1)
# fig.add_trace(go.Scatter(x=bin_df['bin_centres'],
#                          y=bin_df['point-shear-b_bin_mean'],
#                          mode='markers+lines',
#                          name='Mean +/- STD',
#                          line=dict(color='black'),
#                          opacity=.55,
#                          error_y_array=bin_df['point-shear-b_std'],
#                          showlegend=False),
#               row=2, col=1)


fig.add_trace(go.Scatter(x=resid_dict['point-tau_vm-c.out']['Time Step'],
                         y=resid_dict['point-tau_vm-c.out']['point-tau_vm-c'],
                         mode='markers',
                         opacity=scatter_opacity,
                         name='Shear-C',
                         line=dict(color=color10_16[2]),
                         showlegend=True),
              row=3, col=1)
# fig.add_trace(go.Scatter(x=bin_df['bin_centres'],
#                          y=bin_df['point-shear-c_bin_mean'],
#                          mode='markers+lines',
#                          name='Mean +/- STD',
#                          line=dict(color='black'),
#                          opacity=.55,
#                          error_y_array=bin_df['point-shear-c_std'],
#                          showlegend=False),
#               row=3, col=1)

fig.add_trace(go.Scatter(x=resid_dict['point-tau_vm-d.out']['Time Step'],
                         y=resid_dict['point-tau_vm-d.out']['point-tau_vm-d'],
                         mode='markers',
                         opacity=scatter_opacity,
                         name='Shear-D',
                         line=dict(color=color10_16[3]),
                         showlegend=True),
              row=4, col=1)
# fig.add_trace(go.Scatter(x=bin_df['bin_centres'],
#                          y=bin_df['point-shear-d_bin_mean'],
#                          mode='markers+lines',
#                          name='Mean +/- STD',
#                          line=dict(color='black'),
#                          opacity=.55,
#                          error_y_array=bin_df['point-shear-d_std'],
#                          showlegend=False),
#               row=4, col=1)

fig.add_trace(go.Scatter(x=resid_dict['point-tau_vm-e.out']['Time Step'],
                         y=resid_dict['point-tau_vm-e.out']['point-tau_vm-e'],
                         mode='markers',
                         opacity=scatter_opacity,
                         name='Shear-E',
                         line=dict(color=color10_16[4]),
                         showlegend=True),
              row=5, col=1)
# fig.add_trace(go.Scatter(x=bin_df['bin_centres'],
#                          y=bin_df['point-shear-e_bin_mean'],
#                          mode='markers+lines',
#                          name='Mean +/- STD',
#                          line=dict(color='black'),
#                          opacity=.55,
#                          error_y_array=bin_df['point-shear-e_std'],
#                          showlegend=True),
#               row=5, col=1)

fig.update_layout(height=700, width=500,
                  title_text="Von Mises Stress Monitors")

fig.show()

In [26]:

fig = go.Figure()
fig.add_trace(go.Scatter(x=resid_dict['Pressure_Head']['Time Step'],
                         y=resid_dict['Pressure_Head']['p-head-mmHg'],
                         mode='markers',
                         opacity=scatter_opacity,))
# fig.add_trace(go.Scatter(x=bin_df['bin_centres'],
#                          y=bin_df['p-in_bin_mean'],
#                          mode='markers+lines',
#                          name='Mean +/- STD',
#                          line=dict(color='black'),
#                          opacity=.55,
#                          error_y_array=bin_df['p-in_std'],
#                          showlegend=True))
fig.update_layout(
    title="Area Weighted Average Pressure Head",
    xaxis_title="Iteration",
    yaxis_title="Pressure (mmHg)",
    legend_title=None,height=300, width=700,)
fig.show()
print('Time Averaged Pressure Head (mmHg) each blade pass: ' + str(time_sample_dict['Pressure_Head']['p-head-mmHg'].mean()))


Time Averaged Pressure Head (mmHg) each blade pass: 284.3971587418732


In [27]:


fig = make_subplots(
    rows=2,
    cols=1,
    shared_xaxes=True,
    x_title="Time Step",
    y_title="pfHb/Hb ",subplot_titles=('GW pfHb/Hb', 'HO pfHb/Hb') )

fig.add_trace(go.Scatter(x=resid_dict['gw-hemolysis.out']['Time Step'],
                         y=resid_dict['gw-hemolysis.out']['pfHb/Hb'],
                         mode='markers',
                         opacity=scatter_opacity,),
              row=1,col=1)
# fig.add_trace(go.Scatter(x=bin_df['bin_centres'],
#                          y=bin_df['eds-hemolysis_bin_mean'],
#                          mode='markers+lines',
#                          name='Mean +/- STD',
#                          line=dict(color='black'),
#                          opacity=.55,
#                          error_y_array=bin_df['eds-hemolysis_std'],
#                          showlegend=True))

fig.add_trace(go.Scatter(x=resid_dict['ho-hemolysis.out']['Time Step'],
                         y=resid_dict['ho-hemolysis.out']['pfHb/Hb'],
                         mode='markers',
                         opacity=scatter_opacity,),
              row=2,col=1)
fig.update_layout(
    title="Hemolysis Outlet Mass Weighted Average",
    legend_title=None,
    yaxis = dict(showexponent = 'all',
                 exponentformat = 'E'),
    yaxis2 = dict(showexponent = 'all',
                 exponentformat = 'E'))
fig.show()

print('Time Averaged HO pfHb (mg/dL) each blade pass at Outlet: ' + str(time_sample_dict['ho-hemolysis.out']['pfHb'].mean()))
print('Time Averaged GW pfHb (mg/dL) each blade pass at Outlet: ' + str(time_sample_dict['gw-hemolysis.out']['pfHb'].mean()))

Time Averaged HO pfHb (mg/dL) each blade pass at Outlet: 0.12244304751691559
Time Averaged GW pfHb (mg/dL) each blade pass at Outlet: 0.2547005642424348
